# 3.1 신경망의 구조
- 네트워크(모델)을 구성하는 층
- 입력데이터와 타깃
- 손실함수: 학습에 사용할 피드백 신호를 정의
- 옵티마이저: 학습 진행방식 결정



#### 3.1.1 층 호완성

층마다 적절한 텐서 포맷과 데이터 처리 방식이 다름.
- 2D tensor : 보통 밀집 연결 층(dense connected layer) 으로 처리 (ex.케라스 에서는 DENSE)
- 3D tensor : 순환층(recurrent layer) (ex. LSTM)
- 4D tensor : 2D 합성곱(convolution layer) ( ex. Conv2D)


--> keras의 층 호환성: 입력 크기를 자동으로 상위 층의 크기에 맞추어 결정한다.

In [3]:
from keras import layers
layer = layers.Dense(32, input_shape=(784,)) #32개의 유닛으로 된 밀집 층

In [4]:
from keras import models
from keras import layers

model=models.Sequential()
model.add(layers.Dense(32, input_shape=(784,)))
model.add(layers.Dense(10))

3.2.2 케라스 사용
- 모델 정의 방법 두 가지: Sequential 클래스, 함수형 API(7장)

In [5]:
# 1) Sequential 클래스
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10,activation='softmax'))

In [8]:
# 2) 같은 모델의 함수형 API

input_tensor=layers.Input(shape=784,)
x=layers.Dense(32, activation='relu')(input_tensor)
output_tensor=layers.Dense(10,activation='softmax')(x)

model=models.Model(inputs=input_tensor, outputs=output_tensor)

In [14]:
# 모델구조 정의 후에는 동일 (Sequential 클래스, 함수형 API 모두 동일)

from keras import optimizers
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='mse',
              metrics=['accuracy'])

# 타겟 데이터(numpy배열)을 모델의 fit()에 전달
model.fit(input_tensor, target_tensor, batch_size=128, epochs=18) 

NameError: ignored

## 3.4 영화 리뷰 분류: 이진 분류

In [16]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


In [18]:
train_data[0] # 각 리뷰는 단어 인덱스의 리스트 (단어 시퀀스가 인코딩된 것)

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [20]:
train_labels

array([1, 0, 0, ..., 0, 1, 0])

In [22]:
word_index=imdb.get_word_index()  #단어와 정수 인덱스 매핑
reverse_word_index=dict([(value,key) for (key, value) in word_index.items()]) #정수인덱스와 단어를 매핑하도록 뒤집음
decoded_review=' '.join([reverse_word_index.get(i-3,'?')for i in train_data[0]]) #리뷰를 디코딩

1646592/1641221 [==============================] - 0s 0us/step


3.4.2 데이터 준비
숫자 리스트를 텐서로 변경.

1. 같은 길이가 되도록 리스트에 페딩 추가. (samples, sequence_length)크기의 정수 텐서로 변환. 
이 정수 텐서를 다룰 수 있는 층을 신경망의 첫 번째 층으로 사용

2. 리스트를 one-hot-encoding. 0과 1의 벡터로 변환.

In [24]:
## 데이터 준비
# one-hot-encoding

import numpy as np
def vectorize_sequences(sequences,dimension=10000):
  results=np.zeros((len(sequences), dimension)) # (len(sequensces), dimension) 크기의 모든 원소가 0인 행렬.
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1. #results[i]에서 특정 인덱스의 위치를 1로 만듦.
  return results

# 벡터로 변환
x_train=vectorize_sequences(train_data)
y_train=vectorize_sequences(test_data)

In [25]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [26]:
# 레이블을 쉽게 벡터로 바꿈.

y_train=np.asarray(train_labels).astype('float32')
y_test=np.asarray(test_labels).astype('float32')

In [30]:
# 신경망 모델 생성
# relu: 음수를 0으로 만드는 함수

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [31]:
# 손실함수와 옵티마이저

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

3.4.4 훈련 검증

In [34]:
# validation dataset 준비

x_val=x_train[:10000]
partial_x_train=x_train[10000:]

y_val=y_train[:10000]
partial_y_train=y_train[10000:]

In [35]:
# 모델 훈련

history=model.fit(partial_x_train,
                  partial_y_train,
                  epochs=20,
                  batch_size=512,
                  validation_data=(x_val,y_val))

Epoch 1/20
30/30 [==============================] - 1s 49ms/step - loss: 0.5146 - accuracy: 0.7885 - val_loss: 0.3830 - val_accuracy: 0.8716
Epoch 2/20
30/30 [==============================] - 1s 36ms/step - loss: 0.3069 - accuracy: 0.9032 - val_loss: 0.3028 - val_accuracy: 0.8890
Epoch 3/20
30/30 [==============================] - 1s 36ms/step - loss: 0.2242 - accuracy: 0.9272 - val_loss: 0.2844 - val_accuracy: 0.8895
Epoch 4/20
30/30 [==============================] - 1s 36ms/step - loss: 0.1773 - accuracy: 0.9421 - val_loss: 0.2904 - val_accuracy: 0.8860
Epoch 5/20
30/30 [==============================] - 1s 36ms/step - loss: 0.1430 - accuracy: 0.9549 - val_loss: 0.2811 - val_accuracy: 0.8877
Epoch 6/20
30/30 [==============================] - 1s 35ms/step - loss: 0.1185 - accuracy: 0.9633 - val_loss: 0.3286 - val_accuracy: 0.8796
Epoch 7/20
30/30 [==============================] - 1s 36ms/step - loss: 0.0978 - accuracy: 0.9694 - val_loss: 0.3174 - val_accuracy: 0.8808
Epoch 8/20
30